In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd

# Khởi tạo MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1,
                                  min_detection_confidence=0.5)

# Khởi tạo MediaPipe Drawing
mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

# Khởi tạo webcam
cap = cv2.VideoCapture(0)

# Cảm xúc mà bạn muốn thu thập dữ liệu
emotions = ["vui ve", "buon ba", "ngac nhien"]
data = []

print("Bắt đầu thu thập dữ liệu...")

for emotion in emotions:
    print(f"\nHãy thể hiện cảm xúc: {emotion}")
    input("Nhấn Enter để bắt đầu thu thập dữ liệu...")

    num_samples = 0
    while num_samples < 50:  # Thu thập 50 mẫu cho mỗi cảm xúc
        ret, frame = cap.read()
        if not ret:
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(frame_rgb)

        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                # Lấy tọa độ landmark và chuẩn hóa
                landmarks = np.array([[lm.x, lm.y, lm.z] for lm in face_landmarks.landmark]).flatten()

                # Lưu landmark và nhãn
                data.append(np.append(landmarks, emotion))
                num_samples += 1

                # Vẽ landmark lên khung hình để theo dõi
                mp_drawing.draw_landmarks(
                    image=frame,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_TESSELATION,
                    landmark_drawing_spec=drawing_spec,
                    connection_drawing_spec=drawing_spec
                )

        cv2.putText(frame, f"Emotion: {emotion} | Samples: {num_samples}/50", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.imshow("Data Collection", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Lưu dữ liệu vào file CSV
df = pd.DataFrame(data)
df.to_csv("emotion_landmarks.csv", index=False)
print("\nThu thập dữ liệu hoàn tất và đã lưu vào emotion_landmarks.csv")

cap.release()
cv2.destroyAllWindows()


Bắt đầu thu thập dữ liệu...

Hãy thể hiện cảm xúc: vui ve

Hãy thể hiện cảm xúc: buon ba

Hãy thể hiện cảm xúc: ngac nhien

Thu thập dữ liệu hoàn tất và đã lưu vào emotion_landmarks.csv


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import pickle

# Đọc dữ liệu từ file CSV
df = pd.read_csv("emotion_landmarks.csv")

# Tách dữ liệu và nhãn
X = df.iloc[:, :-1]  # Tất cả các cột trừ cột cuối cùng
y = df.iloc[:, -1]   # Cột cuối cùng là nhãn

# Chia dữ liệu thành tập huấn luyện và tập kiểm thử
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Khởi tạo và huấn luyện mô hình SVM
model = SVC(kernel='linear', C=1.0, probability=True)
model.fit(X_train, y_train)

# Đánh giá mô hình
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Độ chính xác của mô hình: {accuracy:.2f}")

# Lưu mô hình đã huấn luyện
with open("emotion_model.pkl", "wb") as f:
    pickle.dump(model, f)
print("Mô hình đã được huấn luyện và lưu vào emotion_model.pkl")

Độ chính xác của mô hình: 1.00
Mô hình đã được huấn luyện và lưu vào emotion_model.pkl


In [3]:
import cv2
import mediapipe as mp
import numpy as np
import pickle

# Tải mô hình đã được huấn luyện
with open("emotion_model.pkl", "rb") as f:
    model = pickle.load(f)

# Khởi tạo MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, min_detection_confidence=0.5)

# Khởi tạo MediaPipe Drawing
mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

# Khởi tạo webcam
cap = cv2.VideoCapture(0)

print("Bắt đầu phân loại cảm xúc trong thời gian thực...")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(frame_rgb)

    emotion_text = "None"
    
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Lấy tọa độ landmark và chuẩn hóa
            landmarks = np.array([[lm.x, lm.y, lm.z] for lm in face_landmarks.landmark]).flatten().reshape(1, -1)

            # Dự đoán cảm xúc
            prediction = model.predict(landmarks)
            emotion_text = prediction[0]

            # Vẽ landmark lên khung hình
            mp_drawing.draw_landmarks(
                image=frame,
                landmark_list=face_landmarks,
                connections=mp_face_mesh.FACEMESH_TESSELATION,
                landmark_drawing_spec=drawing_spec,
                connection_drawing_spec=drawing_spec
            )

    cv2.putText(frame, f"Emotion: {emotion_text}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow("Real-time Emotion Classification", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Bắt đầu phân loại cảm xúc trong thời gian thực...


c:\Users\PC\Deskop\Hand_Tracking_DarkFromAbyss\myenv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
c:\Users\PC\Deskop\Hand_Tracking_DarkFromAbyss\myenv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
c:\Users\PC\Deskop\Hand_Tracking_DarkFromAbyss\myenv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
c:\Users\PC\Deskop\Hand_Tracking_DarkFromAbyss\myenv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
c:\Users\PC\Deskop\Hand_Tracking_DarkFromAbyss\myenv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but S